# ONIOM
## A use case for a water absorption problem

## Table of contents:
* [1. Introduction](#1)
* [2. Use case - water absorption](#2)
>* [2.1 Why water absorption is a relevant scientific problem?](#21)
>* [2.2 Why using ONIOM?](#22)
>* [2.3 How is it done?](#23)
>* [2.4 Potential energy scan for an hydrogen bond](#24)
* [3. Closing words](#3)


## 1. Introduction <a class="anchor" id="1"></a>
This notebook is intended to provide an overview of our implementation of the ONIOM method ([Chem. Rev. 2015, 115, 12, 5678–5796](https://pubs.acs.org/doi/10.1021/cr5004419)), a hybrid QM/MM technique. The ONIOM method enables the user to leverage the precision of expensive computational chemistry tools, only where expressly necessary.

When studying large molecules, and their interactions with other systems, use of expensive computational techniques is often not only intractable, but also unnecessary. For example, when considering the interaction between a water molecule and a functional group appended to a large graphite flake, it is not reasonable to attempt to treat this entire problem at the level of something like Full Configuration Interaction (FCI) or Coupled Cluster methods (ex: CCSD(T)). In such situations, the ONIOM method enables us to target the fragments of the problem at hand with the most sophisticated techniques we can afford, while relegating the rest of the problem to the domain of more affordable computational methods. This method can become particularly advantageous when we are interested in energy differences, where the contribution to the total molecular energy from our relatively inert volumes become negligible. 

Formally, the energy evaluated by ONIOM (with a low and high precision method) is expressed as:

\begin{equation}
E_{ONIOM} = E_{All}^{Low} + \sum_{i=1}^N (E_{Fragment\ i}^{High} - E_{Fragment\ i}^{Low})
\end{equation}

Where $E_{All}$, $E_{Fragment}$ are respectively the energy of the whole system and the energy of a fragment. The general procedure for ONIOM is as follows. The user identifies a system of interest. A low-cost method is used to compute the total energy of the system. Subsequently, a subset of the molecule is defined as a model fragment. The fragment is isolated and an atom (or a functional group) is added to keep the valence fully populated. Its energy is computed using both the low-cost method as above and a high-cost method which can be applied on this reduced system. The difference in energy between these two models is then added to our total energy. In this way, we can interpret the ONIOM method as an iterative procedure where the error associated with our low-cost solver is removed.

The equation above is formulated to allow us to expand ONIOM beyond a single fragment. In principle, many such fragments can be defined to progressively improve upon our result. This may apply to the situation where we have more than one active site on our large molecule, or where an incremental strategy can be utilized to further mitigate against errors associated with using our low-cost method in the vicinity of our active region.

## 2. Use case - water absorption<a class="anchor" id="2"></a>

### 2.1 Why water absorption is a relevant scientific problem?<a class="anchor" id="21"></a>

New technologies are in need of scientific insights on the interaction of materials with water. As illustrated below, many applications would benefit of those insights. The first example comes from the water scarcity in some regions of the world. One explored way of circumventing this issue is to construct a device capable of condensing water in the air into a liquid. This requires materials able to attract water molecules, like a magnet, even at low humidity. Another predominant problem for water accessibility comes from its purification. If a material is able to selectively retain water molecules and no other chemical entity, one can perform purification and thus converting saline water into drinkable water. Lastly, interaction of water with the construction of buildings and bridges can potentially have a big impact on investment. Long-term durability of the materials used could be a significant interest in the scientific community.

![Water absorption problems](img/water_absorption_problems.png "Water absorption problems")

All of those problems can be apprehended from a bottom-up perspective with simulation studies. New material discoveries are thus accelerated by insights provided by material modeling. In the notebook, we tackle a specific problem in water absorption with the use of ONIOM.

### 2.2 Why using ONIOM? <a class="anchor" id="22"></a>

During a computational study, one can decide to scale up the system size for extending the possible outcomes of the simulations. In the water absorption study, a single molecule can be positioned near a chemical group to identify the interaction force and optimal heteroatom-hydrogen distance. Scaling up in this case can mean adding more water molecules, thus disabling the hydrogen bond by stabilizing the relevant water molecule.

Even if the system size is very far from the thermodynamic limit, computational resources are still a constrain. Using a problem decomposition procedure can enable those simulations. With the help of ONIOM, one can target a specific interaction to compute with a high accuracy method while considering an environment with a lighter electronic structure solver. This addition of a chemical environment can lead to an easier link between simulation and experimental results while preserving practicable compute time.

### 2.3 How is it done? <a class="anchor" id="23"></a>

For the remaining part of this section, we will see how to combine the Variational Quantum Eigensolver (VQE) with Hartree-Fock (HF) to compute the ground state energy of a water/carboxylic acid system. We will focus our attention on the hydrogen bond between the COOH group and a water molecule.

We want to specifically target the H2O-COOH interaction. As partially stated earlier, we will use the Variational Quantum Eigensolver (VQE) with a Unitary Coupled-Cluster Single and Double excitation (UCCSD) ansatz. The basis set for this fragment is 6-31G(d,p). The rest of the system is computed using Hartree-Fock (HF) with a minimal basis set (STO-3G). Atom numbering has been printed with the relevant atom to include in the fragment. Internally, the `ONIOMProblemDecomposition` solver adds a hydrogen atom between atoms 0 and 1 to preserve the valence bond completeness.

<img src="img/ONIOM_example.png" width="900">

In [ ]:
import math

from qsdk.problem_decomposition.oniom.oniom_problem_decomposition import ONIOMProblemDecomposition
from qsdk.problem_decomposition.oniom._helpers.helper_classes import Fragment, Link
from qsdk.electronic_structure_solvers.vqe_solver import Ansatze

The first thing to do is to define a geometry and some options for the wanted solvers.

As a side note, the quantum resources for this subsystem are still too demanding for NISQ devices. Reduction of resources is then made by freezing orbitals in the system: in our simple case, the fragment has been reduced to a HOMO-LUMO problem. This enables the use of UCC3 as the ansatz, as it is equivalent to UCCSD, shallower (it removes redundant excitation terms) and can be applied to two-level systems. As devices are developed, losing this condition could improve the precision and make the computation more challenging.

In [ ]:
# Coordinates file
with open("xyz/water_system.xyz",'r') as f:
    xyz = f.read()
    # Removing first 2 lines (number of atoms and a comment line)
    xyz = xyz.split("\n", 2)[2]

options_low = {"basis": "sto-3g"}
options_high = {"basis": "6-31G**", 
                "qubit_mapping": "jw", 
                "ansatz": Ansatze.UCC3, 
                "up_then_down": True,
                "frozen_orbitals": [i for i in range(76) if i not in (16, 17)]}

In the next cell, we will construct the `ONIOMProblemDecomposition` object from a multi-line string taken from a coordinate file and `Fragment` objects. Moreover, in our case, a chemical bond is broken: a `Link` object is also defined by passing the atom ids defining the broken bond.

In [ ]:
# Whole system to be computed with a low precision method (RHF, sto-3g)
system = Fragment(solver_low="rhf", options_low=options_low, charge=0)

# Fragment to be computed with a high precision method (VQE-UCC3, 6-31G**).
link = [Link(0, 1, 0.709, 'H')]
model = Fragment(solver_low="rhf",
                 options_low=options_low,
                 solver_high="vqe",
                 options_high=options_high,
                 selected_atoms=[1, 2, 3, 4, 8, 9, 10],
                 broken_links=link,
                 charge=0)

# Construction of the ONIOM solver.
oniom_solver = ONIOMProblemDecomposition({"geometry": xyz, "fragments": [system, model], "verbose": True})

Quantum resources are obtained by calling the `get_resources` method.

In [ ]:
resources = oniom_solver.get_resources()

The last thing to call is the `simulate` method to get the ONIOM energy.

In [ ]:
e_oniom = oniom_solver.simulate()
print("ONIOM Energy: ", e_oniom)

### 2.4 Potential energy scan for an hydrogen bond <a class="anchor" id="24"></a>

The ONIOM energy computed in the last section can be difficult to compare with known validated data. In this case, it is appropriate to get relative results, extract a behavior of them and compare experiments. The hydrogen bond distance is one example of a quantity that can be compared with tabulated data.

One way of getting this distance is to perform an energy scan on many configurations. The water molecule - carboxylic acid distance was changed from 1.0 to 3.5 angstrom. An energy calculation, with the same parameters as in the last section, was performed for each of those distance. The minimal energy point is the ideal hydrogen bond distance for this system.

The computation has already been done to save some time. Here are the results plotted in a graph.

In [ ]:
import matplotlib.pyplot as plt

distances = [3.550862922159438, 3.460762889964292, 3.370670305757595, 3.280585783157128, 3.1905100050249016, 3.1004437335081567, 3.0103878218778832, 2.9203432285513977, 2.8303110337833894, 2.740292459636501, 2.650288894007595, 2.5603019197028303, 2.4703333498422166, 2.3803852712588354, 2.29046009807724, 2.20056063836393, 2.1106901777200746, 2.020852585050676, 1.9310524476726847, 1.8412952456882081, 1.7515875795688893, 1.6619374708395822, 1.572354764687664, 1.4828516770284208, 1.3934435500122229, 1.3041499143311708, 1.214996013163829, 1.126015038057959, 1.0372514935327881, 0.9487664078760378, 0.8606456724847144]
energies = [-1278.051959731282, -1278.057229408819, -1278.0643608792661, -1278.073141506658, -1278.0833167826524, -1278.0945967737384, -1278.1066656511987, -1278.119193003125, -1278.1316621874737, -1278.1441348500723, -1278.1561527375766, -1278.1674486239667, -1278.1778145272128, -1278.1870878716404, -1278.1951499219097, -1278.201919599394, -1278.2073419570033, -1278.2113710433855, -1278.2139489568763, -1278.214985604411, -1278.2143431181898, -1278.21182121378, -1278.2071290776098, -1278.19982995337, -1278.1892617407693, -1278.1744445322772, -1278.153955355375, -1278.1257011700952, -1278.086488264795, -1278.0312418159333, -1277.95164876962]
plt.figure(figsize=(8,4.5))
plt.title("Potential energy scan for an hydrogen bond")
plt.xlabel("O-H distance / Angstrom")
plt.ylabel("Energy / Hartree")
plt.scatter(distances, energies)

The minimum in energy corresponds to about 1.7 $\overset{\circ}{A}$ for the O-H distance. Hydrogen bond distance is known to be between 1.6 and 2.0 $\overset{\circ}{A}$ in bulk water, so this is in fact a plausible result. The bonding energy seems to be well beyond the one expected to a hydrogen bond: this data does not take into account the geometry optimization needed at each point to get relevant bonding energies.

## 3. Closing words<a class="anchor" id="3"></a>

In this notebook, a use case simulation study has been introduced for the water absorption problem. We learn that ONIOM problem decomposition method can be useful for scaling up system size in simulations. This is done by targeting an important interaction in the system, thus allocating more computational (classical or quantum) resources to it. The environment (the remaining part), can be described with a lower accuracy method. Therefore, a simple example has been presented as a potential starting point for a more complete research project.